### Model Creation

In [82]:
using Gmsh

# Define dimensions
L = 1000;           # length in mm
W = 250;            # width in mm
H = 200;            # height in mm
h = 10;            # meshing

# Initialize Gmsh
gmsh.initialize()

# Create a new model
gmsh.model.add("Cantilever_Q5")

# Define corner points
p1 = gmsh.model.geo.addPoint(0, 0, 0, h)
p2 = gmsh.model.geo.addPoint(W, 0, 0, h)
p3 = gmsh.model.geo.addPoint(W, H, 0, h)
p4 = gmsh.model.geo.addPoint(0, H, 0, h)

# Bottom face lines
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

# Curve loop and plane surface for bottom
cl = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
s = gmsh.model.geo.addPlaneSurface([cl])

# Extrude to create 3D 
# ext = gmsh.model.geo.extrude([(dim, tag)], dx, dy, dz)
ext = gmsh.model.geo.extrude([(2, s)], 0, 0, L)

# Extract the extruded entities 
# Correct entity extraction
top_face = ext[1][2]
side1    = ext[2][2]
side2    = ext[3][2]
side3    = ext[4][2]
side4    = ext[5][2]
volume   = ext[6][2]

# Synchronize geometry
gmsh.model.geo.synchronize()

# Entire 3D domain
gmsh.model.addPhysicalGroup(3, [volume], 1)
gmsh.model.setPhysicalName(3, 1, "Domain")

# Fixed face (left end at x = 0)
gmsh.model.addPhysicalGroup(2, [s], 2)
gmsh.model.setPhysicalName(2, 2, "FixedEnd")

# Free End/Face
gmsh.model.addPhysicalGroup(2, [top_face], 3)
gmsh.model.setPhysicalName(2, 3, "LoadFace")

# Generate 3D mesh
gmsh.model.mesh.generate(3)

# Create the folder (if it doesn't exist)
write_dir = "../../../../Model/Q_5"
isdir(write_dir) || mkpath(write_dir)  # creates the folder if missing

# Save mesh
gmsh.write(joinpath(write_dir, "Cantilever_Q5.msh"))

#Launch GUI or Finalize (Launch the GUI to visualize the mesh or finalize the Gmsh API:)
# Launch GUI
gmsh.fltk.run()  

# Finalize Gmsh
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 60%] Meshing curve 9 (Line)
Info    : [ 70%] Meshing curve 11 (Line)
Info    : [ 80%] Meshing curve 12 (Line)
Info    : [ 90%] Meshing curve 16 (Line)
Info    : [100%] Meshing curve 20 (Line)
Info    : Done meshing 1D (Wall 0.00114179s, CPU 0.015625s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 13 (Surface, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 17 (Surface, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 21 (Surface, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 25 (Surface, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 26 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wa

In [ ]:
# Testing the extrusion command
println(ext)

Tuple{Int32, Int32}[(2, 26), (3, 1), (2, 13), (2, 17), (2, 21), (2, 25)]


In [ ]:
# Creating Diectory
mesh_name = "Cantilever_Q5.msh"
mesh_file = joinpath(@__DIR__, "..", "..", "..", "..", "Model", "Q_5", mesh_name)
isfile(mesh_file) || error("Path does not exist: $mesh_file")

In [ ]:
# create vtu file for checkup
model_check = DiscreteModelFromFile(mesh_file)
result_path = joinpath(@__DIR__, "..", "..", "..", "..", "Model", "Q_5", "model_check")
isdir(result_path) || mkpath(result_path)
writevtk(model_check, joinpath(result_path, "model_check"));

### Packages used

In [84]:
using Gridap
using GridapGmsh
using LinearAlgebra

### Code

In [ ]:

P = 1000.0;             # N

In [ ]:
# Recalling the model
model = GmshDiscreteModel(mesh_file)

# Inspect physical groups if you need to find boundary names:
println("Physical groups (cell tags): ", model)

Info    : Reading 'c:\Users\IIT BBSR\Desktop\Amiya\CAD_Lab\CAD_Lab_Assignments\A2\Question_5\..\..\..\..\Model\Q_5\Cantilever_Q5.msh'...
Info    : 27 entities
Info    : 1272 nodes
Info    : 2360 elements
Info    : Done reading 'c:\Users\IIT BBSR\Desktop\Amiya\CAD_Lab\CAD_Lab_Assignments\A2\Question_5\..\..\..\..\Model\Q_5\Cantilever_Q5.msh'


UnstructuredDiscreteModel()

In [ ]:
# Create triag/volume measure
degree = 3
Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

GenericMeasure()

In [91]:
# Boundary triangulation for 2D faces
Γ = BoundaryTriangulation(model)
dΓ = Measure(Γ, 2)

GenericMeasure()

### Finite Element Parameters and Space

In [104]:
# FE spaces 
order = 1

reffe = ReferenceFE(lagrangian,Float64,order)
V0 = TestFESpace(model,reffe;conformity=:H1,dirichlet_tags="FixedEnd")

g(x) = 2.0
Ug = TrialFESpace(V0, g)

TrialFESpace()

### Numerical Integration

In [105]:
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

neumanntags = ["LoadFace"]
Γ = BoundaryTriangulation(model,tags=neumanntags)
dΓ = Measure(Γ,degree)

GenericMeasure()

### Weak Form

In [114]:
# Material Parameters 
E = 25000.0             # N/mm^2
ν = 0.2;                # Poissons Ratio

In [115]:
# Derived Lame constants
λ = (E*ν)/((1+ν)*(1-2ν))
μ = E/(2*(1+ν));

In [116]:
function elasticity_tensor(λ, μ)
    C = zeros(3,3,3,3)
    for i in 1:3, j in 1:3, k in 1:3, l in 1:3
        C[i,j,k,l] = λ*δ(i,j)*δ(k,l) + μ*(δ(i,k)*δ(j,l) + δ(i,l)*δ(j,k))
    end
    return C
end

δ(i,j) = i == j ? 1.0 : 0.0
C = elasticity_tensor(λ, μ)


3×3×3×3 Array{Float64, 4}:
[:, :, 1, 1] =
 27777.8     0.0      0.0
     0.0  6944.44     0.0
     0.0     0.0   6944.44

[:, :, 2, 1] =
     0.0  10416.7  0.0
 10416.7      0.0  0.0
     0.0      0.0  0.0

[:, :, 3, 1] =
     0.0  0.0  10416.7
     0.0  0.0      0.0
 10416.7  0.0      0.0

[:, :, 1, 2] =
     0.0  10416.7  0.0
 10416.7      0.0  0.0
     0.0      0.0  0.0

[:, :, 2, 2] =
 6944.44      0.0     0.0
    0.0   27777.8     0.0
    0.0       0.0  6944.44

[:, :, 3, 2] =
 0.0      0.0      0.0
 0.0      0.0  10416.7
 0.0  10416.7      0.0

[:, :, 1, 3] =
     0.0  0.0  10416.7
     0.0  0.0      0.0
 10416.7  0.0      0.0

[:, :, 2, 3] =
 0.0      0.0      0.0
 0.0      0.0  10416.7
 0.0  10416.7      0.0

[:, :, 3, 3] =
 6944.44     0.0       0.0
    0.0   6944.44      0.0
    0.0      0.0   27777.8

In [117]:
ε(u) = symmetric(∇(u))

a(u,v) = ∫( inner( ε(v), C ⊙ ε(u) ) ) * dΩ
b(v)   = ∫( inner( v, body_force ) ) * dΩ  # if you have body forces

b (generic function with 1 method)

### FE Problem

In [108]:
op = AffineFEOperator(a,b,Ug,V0)

AffineFEOperator()

### Solver

In [109]:
ls = LUSolver()
solver = LinearFESolver(ls)

LinearFESolver()

In [110]:
uh = solve(solver,op)

SingleFieldFEFunction():
 num_cells: 2360
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 10467487683455747748

In [ ]:
result = joinpath(@__DIR__, "..", "..", "..", "..", "Model", "Q_5", "results")
isdir(result) || mkpath(result)

writevtk(Ω, joinpath(result, "model"), cellfields=["uh"=>uh])

(["c:\\Users\\IIT BBSR\\Desktop\\Amiya\\CAD_Lab\\CAD_Lab_Assignments\\A2\\Question_5\\..\\..\\..\\..\\Model\\Q_5\\results\\model.vtu"],)